# Proyecto Integrador — Pipeline incremental de datos

 1 Importar librerías y configurar rutas

In [1]:
# 1) Importar librerías y configurar rutas
import sqlite3
from pathlib import Path
import pandas as pd
import os
import sys

# Rutas (definimos project_root antes de intentar importar desde 'src')
project_root = Path('..')
# Asegurar que la carpeta del proyecto (que contiene 'src') esté en sys.path
# Esto permite importar 'src.proyecto_integrador' desde el notebook
sys.path.insert(0, str(project_root.resolve()))

# Clase de ayuda para descargar/extract (usa kagglehub)
from src.proyecto_integrador.ingestar import Ingestar

db_dir = project_root / 'db'
db_dir.mkdir(parents=True, exist_ok=True)
db_path = db_dir / 'proyecto.db'

csv_dir = project_root / 'csv'
csv_dir.mkdir(parents=True, exist_ok=True)
output_csv = csv_dir / 'export.csv'

# Parámetros
KAGGLE_REF = 'priyamchoksi/credit-card-transactions-dataset'
CHUNK_SIZE = 100_000  # filas por chunk al leer CSV
UNIQUE_ID_COL = 'trans_num'  # columna única para evitar duplicados

 2 Descargar y extraer (si es necesario)

In [2]:
# 2) Descargar y extraer
ing = Ingestar()
dataset_path = ing.download_dataset_zip(KAGGLE_REF)
data_dir = ing.extract_zip_files(dataset_path)

📥 Dataset descargado: C:\Users\eulic\.cache\kagglehub\datasets\priyamchoksi\credit-card-transactions-dataset\versions\1
📄 1 archivo(s) CSV encontrado(s)


In [3]:
# 3) Carga a SQLite — usando función del módulo load_to_sqlite
from src.proyecto_integrador.load_to_sqlite import insertar_datos
import pandas as pd
import os
import sqlite3

# Busca archivos CSV en la carpeta de datos
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
if not csv_files:
    raise FileNotFoundError('No se encontraron archivos CSV en la carpeta de dataset')

# Leer y concatenar todos los CSV en un único DataFrame
dfs = []
for csv_file in csv_files:
    path = os.path.join(data_dir, csv_file)
    print(f'Leyendo {path}...')
    df_tmp = pd.read_csv(path, low_memory=False)
    dfs.append(df_tmp)

df_all = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

if df_all.empty:
    print('No hay filas en los CSV; la tabla no será creada.')
else:
    # Conectar a SQLite y usar la función del módulo
    conn = sqlite3.connect(db_path, timeout=60)
    insertar_datos(conn, df_all)
    conn.close()

Leyendo C:\Users\eulic\.cache\kagglehub\datasets\priyamchoksi\credit-card-transactions-dataset\versions\1\credit_card_transactions.csv...
 1,296,675 registros insertados en la base de datos
 1,296,675 registros insertados en la base de datos


In [4]:
# 4) Exportar tabla completa a CSV (usando función del módulo export_to_csv)
from src.proyecto_integrador.export_to_csv import exportar_tabla_a_csv

exportar_tabla_a_csv(db_path, output_csv)

 1,296,675 registros exportados a ..\csv\export.csv


## 5 Comprobaciones básicas de la base de datos


### 5.1 Listar tablas disponibles

Esta celda lista las tablas presentes en la base de datos SQLite.

In [5]:

try:
    db_path
except NameError:
    db_path = Path('..') / 'db' / 'proyecto.db'
db_path = Path(db_path)
print(f"Usando base de datos: {db_path.resolve()}")

if not db_path.exists():
    raise FileNotFoundError(f'No se encontró la base de datos en {db_path}')

conn = sqlite3.connect(str(db_path))
try:
    tables_df = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';", conn)
    display(tables_df)
finally:
    conn.close()

Usando base de datos: C:\Users\eulic\Documents\Proyecto integrado 5\piv_2025_2_2\db\proyecto.db


,name
0,transacciones


### Conteo de filas por tabla

 calculamos el número de filas de cada tabla para detectar tablas vacías o tamaños inesperados.

In [6]:
try:
    db_path
except NameError:
    db_path = Path('..') / 'db' / 'proyecto.db'
conn = sqlite3.connect(str(db_path))
try:
    tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';", conn)['name'].tolist()
    counts = []
    for t in tables:
        c = pd.read_sql_query(f'SELECT COUNT(*) AS cnt FROM "{t}";', conn).iloc[0,0]
        counts.append({'table': t, 'rows': c})
    display(pd.DataFrame(counts))
finally:
    conn.close()

,table,rows
0,transacciones,1296675


###  Filas de ejemplo

 mostramos hasta 5 filas por tabla para inspección rápida de datos y tipos.

In [7]:

try:
    db_path
except NameError:
    db_path = Path('..') / 'db' / 'proyecto.db'
conn = sqlite3.connect(str(db_path))
try:
    tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';", conn)['name'].tolist()
    for t in tables:
        print('\n' + '-'*60)
        print(f"Ejemplos de filas para: {t}")
        df = pd.read_sql_query(f'SELECT * FROM "{t}" LIMIT 5;', conn)
        display(df)
finally:
    conn.close()


------------------------------------------------------------
Ejemplos de filas para: transacciones


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,merch_zipcode
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,28705.0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,NaN
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,83236.0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,NaN
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,22844.0


### Nulos por columna
para cada tabla calculamos cuántos valores NULL tiene cada columna; esto ayuda a detectar columnas mayormente vacías.

In [8]:

try:
    db_path
except NameError:
    db_path = Path('..') / 'db' / 'proyecto.db'
conn = sqlite3.connect(str(db_path))
try:
    tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';", conn)['name'].tolist()
    for t in tables:
        print('\n' + '-'*60)
        print(f"Nulos por columna en: {t}")
        schema = pd.read_sql_query(f"PRAGMA table_info('{t}');", conn)
        cols = schema['name'].tolist()
        if not cols:
            print('Sin columnas detectadas')
            continue
        null_expr = ", ".join([f'SUM(CASE WHEN "{c}" IS NULL THEN 1 ELSE 0 END) AS "{c}_nulls"' for c in cols])
        nulls = pd.read_sql_query(f'SELECT {null_expr} FROM "{t}";', conn).T
        nulls.columns = ['nulos']
        display(nulls)
finally:
    conn.close()


------------------------------------------------------------
Nulos por columna en: transacciones


,nulos
Unnamed: 0_nulls,0
trans_date_trans_time_nulls,0
cc_num_nulls,0
merchant_nulls,0
category_nulls,0
amt_nulls,0
first_nulls,0
last_nulls,0
gender_nulls,0
street_nulls,0


###  Conteo de valores únicos en `UNIQUE_ID_COL`

si la variable `UNIQUE_ID_COL` está definida en el notebook y existe en la tabla, calculamos el número de valores distintos y la cantidad de filas duplicadas (si las hay).

In [9]:

# Usar UNIQUE_ID_COL si existe en el notebook
UNIQUE_ID = globals().get('UNIQUE_ID_COL', None)
try:
    db_path
except NameError:
    db_path = Path('..') / 'db' / 'proyecto.db'
conn = sqlite3.connect(str(db_path))
try:
    tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';", conn)['name'].tolist()
    if not UNIQUE_ID:
        print('No se encontró `UNIQUE_ID_COL` definido en el notebook. Define UNIQUE_ID_COL = "col_name" para usar esta comprobación.')
    else:
        for t in tables:
            print('\n' + '-'*60)
            print(f"Tabla: {t}")
            cols = pd.read_sql_query(f"PRAGMA table_info('{t}');", conn)['name'].tolist()
            if UNIQUE_ID in cols:
                total = pd.read_sql_query(f'SELECT COUNT(*) as total FROM "{t}";', conn).iloc[0,0]
                distinct = pd.read_sql_query(f'SELECT COUNT(DISTINCT "{UNIQUE_ID}") as distinct_ids FROM "{t}";', conn).iloc[0,0]
                print(f'Filas totales: {total}')
                print(f'Valores distintos en "{UNIQUE_ID}": {distinct}')
                if total > distinct:
                    print(f'Posibles duplicados: {total - distinct} filas con el mismo "{UNIQUE_ID}"')
            else:
                print(f'La columna "{UNIQUE_ID}" no existe en la tabla {t}.')
finally:
    conn.close()


------------------------------------------------------------
Tabla: transacciones
Filas totales: 1296675
Valores distintos en "trans_num": 1296675
Filas totales: 1296675
Valores distintos en "trans_num": 1296675
